# Advanced Querying Mongo

Importing libraries and setting up connection

In [5]:
%pip install pymongo


     -------------------------------------- 382.5/382.5 kB 2.6 MB/s eta 0:00:00
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)


In [7]:
from pymongo import MongoClient
cursor = MongoClient()

In [21]:
str_conn='mongodb://localhost:27017'
cursor=MongoClient(str_conn)

cursor.list_database_names()

['admin', 'config', 'ironhack', 'local']

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [25]:
 colec = cursor.ironhack.companies

In [28]:
res=list(colec.find())
res[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [34]:
select={'_id':0, 'name':1}
list(colec.find({'name':'Babelgum'}, select))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [44]:
list(colec.find({'number_of_employees': {'$gt': 5000}}).sort('number_of_employees',-1).limit(1))

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'permalink': 'siemens',
  'crunchbase_url': 'http://www.crunchbase.com/company/siemens',
  'homepage_url': 'http://www.siemens.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'Siemens',
  'category_code': 'hardware',
  'number_of_employees': 405000,
  'founded_year': 1847,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'automation, building-technologies, drive-technology, energy',
  'alias_list': '',
  'email_address': 'contact@siemens.com',
  'phone_number': '49 89 636 34134',
  'description': 'Electronics and Electrical Engineering',
  'created_at': 'Thu Jul 31 09:29:43 UTC 2008',
  'updated_at': 'Thu Nov 28 20:32:55 UTC 2013',
  'overview': '<p>Siemens AG, an electronics and electrical engineering company, operates in the industry, energy, and healthcare sectors worldwide. 

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [66]:
query = {'$and': [{'founded_year': {'$gte': 2000}},
                 {'founded_year': {'$lte': 2005}}]}
select={'_id': 0, 'name': 1, 'founded_year': 1}
list(colec.find(query, select).limit(1))

[{'name': 'Wetpaint', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [67]:
query = {'$and': [{'ipo.valuation_amount': {'$gt': 100000000}},
                 {'founded_year': {'$lt': 2010}}]}
select={'_id': 0, 'name': 1, 'ipo': 1}
list(colec.find(query, select).limit(1))

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [68]:
query = {'$and': [{'number_of_employees': {'$lt': 1000}},
                 {'founded_year': {'$lt': 2005}}]}
select={'_id': 0, 'name': 1,}
list(colec.find(query, select).sort('number_of_employees', -1).limit(1))

[{'name': 'Infinera Corporation'}]

### 6. All the companies that don't include the `partners` field.

In [69]:
query = {'partners': {'$exists': False}}
select={'_id': 0, 'name': 1,}
list(colec.find(query, select).limit(1))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [70]:
query = {'category_code': {'$eq': None }}
select={'_id': 0, 'name': 1}
list(colec.find(query, select).limit(1))

[{'name': 'Collective'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [73]:
query = {'$and': [{'number_of_employees': {'$gte': 100}},
                 {'number_of_employees': {'$lt': 1000}}]}
select={'_id': 0, 'name': 1, 'number_of_employees': 1}
list(colec.find(query, select).limit(1))

[{'name': 'AdventNet', 'number_of_employees': 600}]

### 9. Order all the companies by their IPO price in a descending order.

In [81]:
query = {'ipo': {'$exists': True}}
select={'_id': 0, 'name': 1, 'ipo.valuation_amount':1}
list(colec.find(query,select).sort('ipo', -1).limit(1))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [82]:
query = {'number_of_employees': {'$exists': True}}
select={'_id': 0, 'name': 1}
list(colec.find(query,select).sort('number_of_employees', -1).limit(10))

[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [83]:
query = {'founded_month': {'$gte': 7}}
select={'_id': 0, 'name': 1}
list(colec.find(query,select).sort('number_of_employees', -1).limit(10)) 
#debería ser Limit 1000 pero para no putearte Javi

[{'name': 'PayPal'},
 {'name': 'ExxonMobil'},
 {'name': 'Google'},
 {'name': 'kalimatapla'},
 {'name': 'BBC'},
 {'name': 'The Walt Disney Company'},
 {'name': 'eBay'},
 {'name': 'Groupon'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Adobe Systems'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [85]:
query = {'$and': [{'founded_year': {'$lt': 2000}},
                 {'acquisition.price_amount': {'$gt': 10000}}]}
select={'_id': 0, 'name': 1}
list(colec.find(query,select).limit(1))

[{'name': 'Postini'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [92]:
query = {'acquisition.acquired_year': {'$gt': 2010}}
select={'_id': 0, 'name': 1, 'acquisition':1}
list(colec.find(query,select).sort('acquisition.price_amount', -1).limit(1))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [90]:
query = {'founded_year': {'$exists': True}}
select={'_id': 0, 'name': 1, 'founded_year': 1}
list(colec.find(query,select).sort('founded_year', -1).limit(1))

[{'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [91]:
query = {'founded_day': {'$lte': 7}}
select={'_id': 0, 'name': 1}
list(colec.find(query,select).sort('acquisition.price_amount', -1).limit(10))

[{'name': 'Netscape'},
 {'name': 'PayPal'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Postini'},
 {'name': 'Danger'},
 {'name': 'Clearwell Systems'},
 {'name': 'PrimeSense'},
 {'name': 'Amobee'},
 {'name': 'BlueLithium'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [93]:
query = {'$and': [{'category_code': {'$eq': 'web'}},
                 {'number_of_employees': {'$gt': 4000}}]}
select={'_id': 0, 'name': 1}
list(colec.find(query,select).sort('number_of_employees', 1).limit(2))

[{'name': 'Expedia'}, {'name': 'AOL'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [94]:
query = {'$and': [{'ipo.valuation_amount': {'$gt': 10000000}},
                 {'ipo.valuation_currency_code': {'$eq': 'EUR'}}]}
select={'_id': 0, 'name': 1}
list(colec.find(query,select).limit(2))

[{'name': 'Pironet'}, {'name': 'Pironet'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [97]:
query = {'acquisition.acquired_month': {'$lte': 3}}
select={'_id': 0, 'name': 1, 'acquisition': 1}
list(colec.find(query,select).limit(10)) 

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [101]:
query = {'$and': [{'founded_year': {'$gt': 2000}},
                 {'founded_year': {'$lt': 2010}},
                {'$or': [{'acquisition.acquired_year': {'$gt': 2011}},
                        {'acquisition.acquired_year': {'$exists': False}}]}]}
select={'_id': 0, 'name': 1}
list(colec.find(query,select).limit(2))

[{'name': 'Wetpaint'}, {'name': 'Zoho'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [107]:
query={'$expr': {'$gt': ['$deadpooled_year', {'$add': ['$founded_year', 3]}]}}
list(colec.find(query, {'name': 1, '_id': 0, 'deadpooled_year':1}).limit(10))

[{'name': 'Babelgum', 'deadpooled_year': 2013},
 {'name': 'Thoof', 'deadpooled_year': 2013},
 {'name': 'Mercora', 'deadpooled_year': 2008},
 {'name': 'Wesabe', 'deadpooled_year': 2010},
 {'name': 'Stickam', 'deadpooled_year': 2013},
 {'name': 'AllPeers', 'deadpooled_year': 2008},
 {'name': 'EQO', 'deadpooled_year': 2012},
 {'name': 'AllofMP3', 'deadpooled_year': 2007},
 {'name': 'SellABand', 'deadpooled_year': 2010},
 {'name': 'Zlio', 'deadpooled_year': 2011}]